In [1]:
# install GraphRicciCurvature package

!pip install GraphRicciCurvature

# import packages

import networkx as nx
import matplotlib.pyplot as plt
from time import perf_counter
import numpy as np

from scipy.optimize import minimize

from GraphRicciCurvature.FormanRicci import FormanRicci
from GraphRicciCurvature.OllivierRicci import OllivierRicci


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 682 kB 5.2 MB/s 
     |████████████████████████████████| 9.8 MB 26.0 MB/s 


In [2]:
def simple_cycles(G, limit):
    subG = type(G)(G.edges())
    sccs = list(nx.strongly_connected_components(subG))
    while sccs:
        scc = sccs.pop()
        startnode = scc.pop()
        path = [startnode]
        blocked = set()
        blocked.add(startnode)
        stack = [(startnode, list(subG[startnode]))]

        while stack:
            thisnode, nbrs = stack[-1]

            if nbrs and len(path) < limit:
                nextnode = nbrs.pop()
                if nextnode == startnode:
                    yield path[:]
                elif nextnode not in blocked:
                    path.append(nextnode)
                    stack.append((nextnode, list(subG[nextnode])))
                    blocked.add(nextnode)
                    continue
            if not nbrs or len(path) >= limit:
                blocked.remove(thisnode)
                stack.pop()
                path.pop()
        subG.remove_node(startnode)
        H = subG.subgraph(scc)
        sccs.extend(list(nx.strongly_connected_components(H)))

In [3]:
def afr_curvature (G, ni, nj, t_coeff, t_num):
    '''
    computes the Augmented Forman-Ricci curvature of a given edge 
    includes 3-cycles in calculation 
    
    Parameters
    ----------
    G : Graph
    ni : node i
    nj : node j
    m : number of triangles containing the edge between node i and j

    Returns
    -------
    afrc : int
        Forman Ricci curvature of the edge connecting nodes i and j   
    '''
    afrc = 4 - G.degree(ni) - G.degree(nj) + t_coeff * t_num
    return afrc

In [4]:
def afr4_curvature (G, ni, nj, t_coeff, t_num, q_coeff, q_num):
    '''
    computes the Augmented Forman-Ricci curvature of a given edge, 
    includes 3- and 4-cycles in calculation 
    
    Parameters
    ----------
    G : Graph
    ni : node i
    nj : node j
    t : number of triangles containing the edge between node i and j
    q : number of quadrangles containing the edge between node i and j

    Returns
    -------
    afrc4 : int
        enhanced Forman Ricci curvature of the edge connecting nodes i and j   
    '''
    afrc4 = 4 - G.degree(ni) - G.degree(nj) + t_coeff * t_num + q_coeff * q_num
    return afrc4

In [5]:
def afr5_curvature (G, ni, nj, t_coeff, t_num, q_coeff, q_num, p_coeff, p_num):
    '''
    computes the Augmented Forman-Ricci curvature of a given edge 
    includes 3-, 4- and 5-cycles in calculation 
    
    Parameters
    ----------
    G : Graph
    ni : node i
    nj : node j
    t : number of triangles containing the edge between node i and j
    q : number of quadrangles containing the edge between node i and j
    p : number of pentagons containing the edge between node i and j

    Returns
    -------
    afrc5 : int
        enhanced Forman Ricci curvature of the edge connecting nodes i and j   
    '''
    afrc5 = 4 - G.degree(ni) - G.degree(nj) + t_coeff * t_num + q_coeff * q_num + p_coeff * p_num
    return afrc5

In [6]:
def init_edge_attributes(G):
    curv_names = ["frc", "afrc", "afrc4", "afrc5"] 
    for (u,v) in list(G.edges()):
        for i in range(3,6):
            G.edges[u,v][cyc_names[i]] = []
        for cn in curv_names:
            G.edges[u,v][cn] = 0

In [7]:
def set_edge_attributes_2 (G, ll, i):
    for l in ll:     # für jeden Zyklus in der Liste der Zyklen
        for e1 in range(0, i): 
            if e1 == i-1:
                e2 = 0
            else:
                e2 = e1 + 1
            u = l[e1]
            v = l[e2]
            G.edges[u,v][cyc_names[i]].append(l)

In [8]:
def get_within_vs_between_curv(Gr, cmp_key="block"):
    curv_names = ["orc", "frc", "afrc", "afrc4", "afrc5"] 
    
    c_dict = {"withins": {}, "betweens": {}}
    for k in c_dict.keys():
        for cn in curv_names:
            c_dict[k][cn] = {"data": [], "mean": 0, "std": 0}
        
    for u,v,d in Gr.edges.data():                           # für alle Kanten 
        if Gr.nodes[u][cmp_key] == Gr.nodes[v][cmp_key]:    # innerhalb eines Blocks
            for cn in curv_names:                          # für alle verschiedenen Curvartures
                c_dict["withins"][cn]["data"].append(Gr.edges[u,v][cn])           # hängt den Curvature-Wert der aktuellen Kante an die LIste an 
        else:                                               # zwischen Blöcken
            for cn in curv_names:                          # für alle verschiedenen Curvartures
                c_dict["betweens"][cn]["data"].append(Gr.edges[u,v][cn])           # hängt den Curvature-Wert der aktuellen Kante an die LIste an 

    for k in c_dict.keys():
        for cn in curv_names:
            c_dict[k][cn]["mean"] = np.mean(c_dict[k][cn]["data"])      # Mittelwerte berechnen
            c_dict[k][cn]["std"] = np.std(c_dict[k][cn]["data"])        # Std.-abw. berechnen
            
    res_diffs = {}
    for cn in curv_names:
        sum_std = np.sqrt(np.square(c_dict["withins"][cn]["std"]) + np.square(c_dict["betweens"][cn]["std"]))   # Gesamt-Stdabw berechnen
        res_diffs[cn] = np.abs((c_dict["withins"][cn]["mean"] - c_dict["betweens"][cn]["mean"]) / sum_std)     # Differenz der Mittelwerte bilden und normieren
    
    return res_diffs

In [9]:
def get_orc_edge_curvatures (G):          
    # compute the Ollivier-Ricci curvature of the given graph G
    orc = OllivierRicci(G, alpha=0.5, verbose="ERROR")
    orc.compute_ricci_curvature()
    # transfer curvatire values from orc.G to G 
    for (u,v) in list(orc.G.edges()):               # für jede Kante
        G.edges[u,v]["orc"] = orc.G.edges[u,v]["ricciCurvature"]
        # print("ORC: ", orc.G.edges[u,v]["ricciCurvature"], ("  -  G: ",G.edges[u,v]["orc"])

In [10]:
def get_edge_curvatures (G, t_coeff = 3, q_coeff = 2, p_coeff = 1):            
    for (u,v) in list(G.edges()):               # für jede Kante
        tr = len(G.edges[u,v][cyc_names[3]]) / 2  # geteilt durch 2 wegen gerichtetem Graph und daher immer zwei Permutationen pro Zyklus (1x vorwärts / 1x rückwärts)
        qu = len(G.edges[u,v][cyc_names[4]]) / 2
        pe = len(G.edges[u,v][cyc_names[5]]) / 2
        # G.edges[u,v]["frc"] = fr_curvature(G, u, v)        
        G.edges[u,v]["afrc"] = afr_curvature(G, u, v, t_coeff, tr)
        G.edges[u,v]["afrc4"] = afr4_curvature(G, u, v, t_coeff, tr, q_coeff, qu)
        G.edges[u,v]["afrc5"] = afr5_curvature(G, u, v, t_coeff, tr, q_coeff, qu, p_coeff, pe)     

In [11]:
def get_curvature_gap(Gr, cn="afrc", cmp_key="block"):
    c_dict = {"withins": {}, "betweens": {}}
    for k in c_dict.keys():
        c_dict[k] = {"data": [], "mean": 0, "std": 0}
        
    for u,v,d in Gr.edges.data():                           # für alle Kanten 
        if Gr.nodes[u][cmp_key] == Gr.nodes[v][cmp_key]:    # innerhalb eines Blocks
            c_dict["withins"]["data"].append(Gr.edges[u,v][cn])           # hängt den Curvature-Wert der aktuellen Kante an die LIste an 
        else:                                               # zwischen Blöcken
            c_dict["betweens"]["data"].append(Gr.edges[u,v][cn])           # hängt den Curvature-Wert der aktuellen Kante an die LIste an 

    for k in c_dict.keys():
        c_dict[k]["mean"] = np.mean(c_dict[k]["data"])      # Mittelwerte berechnen
        c_dict[k]["std"] = np.std(c_dict[k]["data"])        # Std.-abw. berechnen
            
    sum_std = np.sqrt(np.square(c_dict["withins"]["std"]) + np.square(c_dict["betweens"]["std"]))   # Gesamt-Stdabw berechnen
    curv_gap = np.abs((c_dict["withins"]["mean"] - c_dict["betweens"]["mean"]) / sum_std)     # Differenz der Mittelwerte bilden und normieren
    
    return curv_gap

In [16]:
def build_size_list (k, l):
    ll = [k  for i in range(l)]
    return ll

In [17]:
def build_prob_list (n, p_in, p_out):
    ll = []
    for i in range(n):    
        temp_l = [p_out  for j in range(0,i)] + [p_in] + [p_out  for j in range(i+2,n+1)]
        ll.append(temp_l)
    return ll

In [12]:
cyc_names = {3:"triangles", 4:"quadrangles", 5:"pentagons"}        


In [13]:
def optimization_func(a, cmp_key="value"):
    t = a[0]
    if len(a) > 1:
        q = a[1]
    else:
        q = 0
    if len(a) > 2:
        p = a[2]
    else:
        p = 0
    # print("t=", t, " q=", q, " p=", p)
    get_edge_curvatures(G, t, q, p)
    c_gap = -1 * get_curvature_gap(G, "afrc5", cmp_key)
    # Comment next line to suppress printing
    # print("t=",t," q=",q," p=",p," curv gap:",c_gap)

    return c_gap

In [14]:
def maximize_curvature_gap(a, cmp_key = "value"):
    for i in range(len(a)):
        x0 = a[0:i+1]    # Initial values for t / t,q / t,q,p
        res = minimize(optimization_func, x0, method='nelder-mead', args = (cmp_key), options={'disp': True})
        print("Parameters [t(,q)(,p)]: ",res.x)
        print("Optimized curvature gap: ",-1 * res.fun, "\n")


---
Karate Club Graph



In [ ]:
# ----------------------------------------
# -------  Karate Club graph  ------------
# ----------------------------------------

def calculate_karate_club(title_str):

    G = nx.karate_club_graph()
    init_edge_attributes(G)
    H = G.to_directed()

    # pos1 = nx.kamada_kawai_layout(H)
    # colors= [0  if v["club"] == "Mr. Hi" else 1  for u,v in H.nodes.data()]
    # plot_my_graph(H, pos1, node_col = colors)
    
    cycles = []
    for c in simple_cycles(H, 6):
        cycles.append(c) 
    
    d = dict()
    for i in range(3,6):
        d[i] = [c  for c in cycles  if len(c) == i]
        set_edge_attributes_2(G, d[i], i)
        
    get_orc_edge_curvatures (G)
    get_edge_curvatures(G, t_coeff = 3, q_coeff = 2, p_coeff = 1)
    # show_curv_data (G, title_str, cmp_key="club")

    # res_diffs = get_within_vs_between_curv(G, cmp_key="club")             # normierte Differenz der Curvatures "within" und "between" bestimmen und anzeigen  
    # print("Resulting differences:") 
    # for k,v in res_diffs.items():
    #     print(k.ljust(15,"."), f"{v:8.5f}")
    # print("\n\n")

    return G


G = calculate_karate_club("Karate Club")
maximize_curvature_gap([3,2,1], cmp_key="club")


Optimization terminated successfully.
         Current function value: -0.744230
         Iterations: 21
         Function evaluations: 42
Parameters [t(,q)(,p)]:  [6.08916016]
Optimized curvature gap:  0.744230112391133 

Optimization terminated successfully.
         Current function value: -0.808701
         Iterations: 54
         Function evaluations: 104
Parameters [t(,q)(,p)]:  [13.4659251  -2.02920709]
Optimized curvature gap:  0.8087010488454437 

Parameters [t(,q)(,p)]:  [-4.59704270e+13 -2.57132011e+13  1.32888137e+13]
Optimized curvature gap:  1.2928507337036348 



In [ ]:
# Americal Football (AMF) read-in from football.gml
def calculate_AMF(title_str):
    G = nx.read_gml("football.gml")
    init_edge_attributes(G)
    H = G.to_directed()
    
    # values = set([d["value"]  for n,d in iter(G.nodes.items())])
    # max_value = list(values)[-1]
    # pos_amf = get_circles_in_circle_pos(G, 0.1, 0.6, 0.5, 0.5, max_value, values)
    
    # G = colored_amf_graph(G)
    # plot_my_graph(G, pos_amf, 
    #               node_col = [d["color"]  for n,d in G.nodes.data()],
    #               color_map = "tab20", alpha = 0.7)

    cycles = []
    for c in simple_cycles(H, 6):
        cycles.append(c) 
    
    d = dict()
    for i in range(3,6):
        d[i] = [c  for c in cycles  if len(c) == i]
        set_edge_attributes_2(G, d[i], i)
        
    get_orc_edge_curvatures (G)
    get_edge_curvatures(G, t_coeff = 3, q_coeff = 0.02, p_coeff = 0.002)
    # show_curv_data (G, title_str, cmp_key="value")

    # res_diffs = get_within_vs_between_curv(G, cmp_key="value")             # normierte Differenz der Curvatures "within" und "between" bestimmen und anzeigen  
    # print("Resulting differences:") 
    # for k,v in res_diffs.items():
    #     print(k.ljust(15,"."), f"{v:8.5f}")
    # print("\n\n")

    return G


G = calculate_AMF("American Football League")
maximize_curvature_gap([3,2,1], cmp_key="value")


Parameters [t(,q)(,p)]:  [9.89560465e+11]
Optimized curvature gap:  2.125400741165232 

Optimization terminated successfully.
         Current function value: -2.234468
         Iterations: 46
         Function evaluations: 88
Parameters [t(,q)(,p)]:  [4.2944301  0.86852696]
Optimized curvature gap:  2.234467820990738 

Optimization terminated successfully.
         Current function value: -2.248082
         Iterations: 131
         Function evaluations: 242
Parameters [t(,q)(,p)]:  [4.2102035  0.18830323 0.07405061]
Optimized curvature gap:  2.2480824755121667 



Vorheriger Aufruf, jetzt ersetzt durch Funktion maximize_curvature_gap

In [ ]:
    x0 = np.array([3])    
    res = minimize(optimization_func, x0, method='nelder-mead', args = (cmp_key), options={'disp': True})
    print("Parameters [t(,q)(,p)]: ",res.x)
    print("Optimized curvature gap: ",-1 * res.fun, "\n")

    x0 = np.array([3, 2])    # Initial values for t,q,p
    res = minimize(optimization_func, x0, method='nelder-mead', options={'disp': True})
    print("Parameters [t(,q)(,p)]: ",res.x)
    print("Optimized curvature gap: ",-1 * res.fun, "\n")

    x0 = np.array([3, 2, 1])    # Initial values for t,q,p
    res = minimize(optimization_func, x0, method='nelder-mead', options={'disp': True})
    print("Parameters [t(,q)(,p)]: ",res.x)
    print("Optimized curvature gap: ",-1 * res.fun, "\n")

---
Stochastic Block Model

In [18]:
# --------------------------------
# ----------  simple SBM  --------
# --------------------------------


def simple_SBM():
    sbm = {"size_per_comm" : 20, "num_of_comm" : 4, "p_in" : 0.70, "p_out" : 0.05}
    sizes = build_size_list(sbm["size_per_comm"], sbm["num_of_comm"])
    probs = build_prob_list(sbm["num_of_comm"], sbm["p_in"], sbm["p_out"])
    
    G = nx.stochastic_block_model(sizes, probs, seed=0)   
    init_edge_attributes(G)
      
    H = G.to_directed()
    
    cycles = []                         # hier werden die Zyklen bestimmt
    for c in simple_cycles(H, 6):       # siehe oben: Funktion simple_cycles
        cycles.append(c) 
    
    d = dict()                          
    for i in range(3,6):
        d[i] = [c  for c in cycles  if len(c) == i]       # in d werden die Zyklen sortiert nach Länge
        set_edge_attributes_2(G, d[i], i)                 # und für die Bestimmung der Curvature-Werte genutzt
        
    get_orc_edge_curvatures(G)                            # Alle Curvatures bestimmen
    get_edge_curvatures(G, t_coeff = 3, q_coeff = 0.02, p_coeff = 0.002)
    


    return G

G = simple_SBM()
maximize_curvature_gap([3,2,1], cmp_key="block")


Optimization terminated successfully.
         Current function value: -4.061278
         Iterations: 14
         Function evaluations: 28
Parameters [t(,q)(,p)]:  [3.37895508]
Optimized curvature gap:  4.061278095608505 

Optimization terminated successfully.
         Current function value: -4.828953
         Iterations: 49
         Function evaluations: 98
Parameters [t(,q)(,p)]:  [0.71455462 0.18948533]
Optimized curvature gap:  4.8289533617959375 

Optimization terminated successfully.
         Current function value: -4.892572
         Iterations: 118
         Function evaluations: 211
Parameters [t(,q)(,p)]:  [ 0.60517732  0.35527772 -0.01153976]
Optimized curvature gap:  4.892571555691521 

